# Project 3 - Part 1

## Business problem

- Produce a MySQL database on movies from a subset of IMDB's publicly available dataset.
- Use this database to analyze what makes a movie successful.
- Provide recommendations to the stakeholder on how to make a successful movie.

## Requirements

- Download several files from IMDB’s movie data set.
- Exclude any movie with missing values for genre or runtime.
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre).
- Include only movies that were released 2000 - 2021 (include 2000 and 2021).
- Include only movies that were released in the United States.

## Imports

In [1]:
import pandas as pd
import os

## Load the data

In [2]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [ ]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [ ]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [ ]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

## Inspect and filter the data: Title AKAs

**Data dictionary**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. E.g. "alternative", "dvd", "festival", "tv", etc.
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title

In [ ]:
akas.info()

In [ ]:
akas.head()

In [ ]:
print(f'There are {akas.shape[0]} rows and {akas.shape[1]} columns in the title_akas dataset.')

In [ ]:
## Align datatypes
akas['isOriginalTitle'] = akas['isOriginalTitle'].astype(bool)

In [ ]:
## Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [ ]:
## Keep only US movies
akas = akas[akas['region'] == 'US']

In [ ]:
## Cleaned and filtered data
akas.info()

## Inspect and filter the data: Title Basics

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

In [ ]:
basics.info()

In [ ]:
basics.head()

In [ ]:
print(f'There are {basics.shape[0]} rows and {basics.shape[1]} columns in the title_basics dataset.')

In [ ]:
## Align datatypes
basics['isAdult'] = basics['isAdult'].astype(bool)
basics['startYear'] = pd.to_datetime(basics['startYear'], format='%Y', errors='coerce')
basics['endYear'] = pd.to_datetime(basics['endYear'], format='%Y', errors='coerce')
basics['runtimeMinutes'] = pd.to_numeric(basics['runtimeMinutes'], errors='coerce')

In [ ]:
## Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [ ]:
## Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [ ]:
## Eliminate movies that are null for genre
basics.dropna(subset=['genres'], inplace=True)

In [ ]:
## Keep only titleType==Movie
basics = basics[basics['titleType'] == 'movie']

In [ ]:
## Keep startYear 2000-2022
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] < '2022')]

In [ ]:
## Eliminate movies that include "Documentary" in genre
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [ ]:
## Keep only US movies
us_movies = basics['tconst'].isin(akas['titleId'])
basics = basics[us_movies]

In [ ]:
## Cleaned and filtered data
basics.info()

## Inspect and filter the data: Title Ratings

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

In [ ]:
ratings.info()

In [ ]:
ratings.head()

In [ ]:
print(f'There are {ratings.shape[0]} rows and {ratings.shape[1]} columns in the title_basics dataset.')

In [ ]:
## Replace "\N" with np.nan
ratings.replace({'\\N':np.nan}, inplace=True)

In [ ]:
## Keep only US movies
us_movies = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[us_movies]

In [ ]:
## Cleaned and filtered data
ratings.info()

## Save the data

In [ ]:
## Create a Data folder
os.makedirs('Data/', exist_ok=True) 

In [ ]:
## Save current dataframes to files
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

## Preview saved data

In [ ]:
akas_clean = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas_clean.info()

In [ ]:
basics_clean = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_clean.info()

In [ ]:
ratings_clean = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings_clean.info()